## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esperance,AU,-33.8667,121.9000,67.84,69,4,21.52,clear sky
1,1,Port Elizabeth,ZA,-33.9180,25.5701,74.10,78,20,21.85,few clouds
2,2,Pisco,PE,-13.7000,-76.2167,70.02,78,0,1.14,clear sky
3,3,Rikitea,PF,-23.1203,-134.9692,77.14,78,100,13.44,light rain
4,4,Riohacha,CO,11.5444,-72.9072,81.90,68,47,14.14,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 50


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

user_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Kostyantynivka,UA,48.5277,37.7069,33.04,100,100,4.63,light snow
9,9,Bethel,US,41.3712,-73.4140,41.23,92,100,0.00,light rain
11,11,Te Anau,NZ,-45.4167,167.7167,49.73,94,100,3.15,light rain
26,26,Kodiak,US,57.7900,-152.4072,38.71,93,100,19.57,moderate rain
28,28,Nanortalik,GL,60.1432,-45.2371,30.42,79,100,13.15,snow
30,30,Iwanai,JP,42.9744,140.5089,23.27,82,100,13.24,light snow
37,37,Punta Arenas,CL,-53.1500,-70.9167,46.42,57,100,6.91,overcast clouds
46,46,Ushuaia,AR,-54.8000,-68.3000,46.02,65,100,14.97,overcast clouds
47,47,Ginda,IN,30.6945,78.4932,38.37,98,100,1.88,light rain
48,48,Port Hardy,CA,50.6996,-127.4199,37.58,100,100,12.66,light rain


In [7]:
# 4a. Determine if there are any empty rows.
user_temp_df.count()

City_ID                145
City                   145
Country                145
Lat                    145
Lng                    145
Max Temp               145
Humidity               145
Cloudiness             145
Wind Speed             145
Current Description    145
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
user_temp_df = user_temp_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = user_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kostyantynivka,UA,33.04,light snow,48.5277,37.7069,
9,Bethel,US,41.23,light rain,41.3712,-73.4140,
11,Te Anau,NZ,49.73,light rain,-45.4167,167.7167,
26,Kodiak,US,38.71,moderate rain,57.7900,-152.4072,
28,Nanortalik,GL,30.42,snow,60.1432,-45.2371,
30,Iwanai,JP,23.27,light snow,42.9744,140.5089,
37,Punta Arenas,CL,46.42,overcast clouds,-53.1500,-70.9167,
46,Ushuaia,AR,46.02,overcast clouds,-54.8000,-68.3000,
47,Ginda,IN,38.37,light rain,30.6945,78.4932,
48,Port Hardy,CA,37.58,light rain,50.6996,-127.4199,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   145
Country                145
Max Temp               145
Current Description    145
Lat                    145
Lng                    145
Hotel Name             145
dtype: int64

In [16]:
hotel_df_null = hotel_df.loc[(hotel_df["Hotel Name"] == "")]
hotel_df_null.count()

City                   9
Country                9
Max Temp               9
Current Description    9
Lat                    9
Lng                    9
Hotel Name             9
dtype: int64

In [17]:
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
clean_hotel_df.count()

City                   136
Country                136
Max Temp               136
Current Description    136
Lat                    136
Lng                    136
Hotel Name             136
dtype: int64

In [18]:
# Inspect clean hotel df prior to export
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Kostyantynivka,UA,33.04,light snow,48.5277,37.7069,Мотель
9,Bethel,US,41.23,light rain,41.3712,-73.4140,Courtyard by Marriott Danbury
11,Te Anau,NZ,49.73,light rain,-45.4167,167.7167,Kingsgate Hotel Te Anau
26,Kodiak,US,38.71,moderate rain,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
28,Nanortalik,GL,30.42,snow,60.1432,-45.2371,Hotel Kap Farvel


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))